# AgCl with Br Complex

In [1]:
% setup environment
% set path to equilibrium solver
txt=pwd;
n=length(txt);
rootpath=txt; rootpath=txt(1:n-22);
PHREEQCpath=[rootpath,'/runPHREEQC'];
addpath(PHREEQCpath);
JEQUILIBpath=[rootpath,'/JEQUILIB'];
addpath(JEQUILIBpath);
% clear data and number format
clear; format short e
% turn off warnings
warning off
%graphics_toolkit ("notebook"); 
%graphics_toolkit ("plotly"); 
%graphics_toolkit("gnuplot")

In [2]:
% MODEL USING TABLEAU (different gradients or log versus linear variables)
%graphics_toolkit("notebook")
%model with just inorganic speciation
%geochemical model
% input the constants and solution chemistry

flag1=1; %flag1 for concentration based search vector (1) or for log based search vector (2).
flag2=2; %flag2 for numerical derivatives (2) or analtyical derivatives (1)
%database=[]; 
flag3=0; % flag3 1 for show warnings. 0 no warnings
flag4=0; %if=1 solve tableau one line at a time to get a good initial guess (no solids). really challenging probs only
flag5=0; %if=1 use stored initial guess for including solids

AgT=1e-6; BrT=1e-5; pH=7; ClT=3.16226666e-5;

TOTALS=[AgT BrT ClT]';
TOTALS(TOTALS==0)=1e-16;% get rid of zero values, b/c div by zero error
%TOTALS=[inorganicTOTALS]; %TOTALS=inorganicTOTALS;
pH=pH; pe=20.75-pH; 

tic;
[Ag,Br,Cl,AgBrs,AgCls,MASSERR]=AgClBrtableaumorecomplex(pH,pe,TOTALS,flag1,flag2,flag3,flag4,flag5);
%[Ag,Br,Cl,AgBrs,AgCls,MASSERR]=AgClBrtableaumorecomplex(pH,pe,1,flag1,flag2,flag3,flag4,flag5);
%[Ag]=AgClBrtableaumorecomplex(pH,pe,TOTALS,flag1,flag2,flag3,flag4,flag5)
tableautime=toc;

Agmasserror=MASSERR(1); Brmasserror=MASSERR(2); Clmasserror=MASSERR(3);

X no ITER EXEED no solid logX


In [3]:
% MODEL USING PHREEQC

minerals=[{'AgBrs'};{'AgCls'}]; totalvector=[AgT; BrT; ClT; ClT+BrT]; totalnames=[{'Ag'}; {'Br'}; {'Cl'}; {'Na'}]; speciesexport=[{'Ag+'}; {'Br-'};{'Cl-'};{'AgBr'};{'AgBr2-'};{'AgBr3-2'};{'AgBr4-3'};{'AgCl'};{'AgCl2-'};{'AgCl3-2'};{'AgCl4-3'}];
%database=['llnl_nosolubleAgCl.dat']; 
acid=['NaOH']; show=0; T=25;
%database=['AgCl.dat']; 
database=['complexAgClBr.dat']; 

tic
[solutionspeciesconcs, speciesnames, solidconcs, solidnames]=runPHREEQCv2noHA(T,pH,pe,totalnames,totalvector,minerals,speciesexport,database,show,acid);
PHREEQCtime=toc

AgPHREEQC=solutionspeciesconcs(1); 
BrPHREEQC=solutionspeciesconcs(2); 
ClPHREEQC=solutionspeciesconcs(3);
AgBrPHREEQC=solutionspeciesconcs(4); 
AgBr2PHREEQC=solutionspeciesconcs(5); 
AgBr3PHREEQC=solutionspeciesconcs(6); 
AgBr4PHREEQC=solutionspeciesconcs(7);
AgClPHREEQC=solutionspeciesconcs(8);
AgCl2PHREEQC=solutionspeciesconcs(9);
AgCl3PHREEQC=solutionspeciesconcs(10);
AgCl4PHREEQC=solutionspeciesconcs(11);
AgBrsPHREEQC=solidconcs(1)
AgClsPHREEQC=solidconcs(2)

AgmasserrorPHREEQC=AgT-AgPHREEQC-AgBrsPHREEQC-AgBrPHREEQC-AgBr2PHREEQC-AgBr3PHREEQC-AgBr4PHREEQC 
BrmasserrorPHREEQC=BrT-BrPHREEQC-AgBrsPHREEQC-AgBrPHREEQC-2*AgBr2PHREEQC-3*AgBr3PHREEQC-3*AgBr4PHREEQC
ClmasserrorPHREEQC=ClT-ClPHREEQC-AgClsPHREEQC-AgClPHREEQC-2*AgCl2PHREEQC-3*AgCl4PHREEQC-3*AgCl4PHREEQC

PHREEQCtime = 3.0581e-02


AgBrsPHREEQC = 9.2027e-07


AgClsPHREEQC =          0


AgmasserrorPHREEQC = 3.4781e-09


BrmasserrorPHREEQC = 4.2848e-19


ClmasserrorPHREEQC = -3.3340e-10


In [4]:
% compare models

Agcompare=[Ag AgPHREEQC]
Brcompare=[Br BrPHREEQC]
Clcompare=[Cl ClPHREEQC]
AgBrcompare=[AgBrs AgBrsPHREEQC]
AgClcompare=[AgCls AgClsPHREEQC]
Agmasserrorcompare=[Agmasserror AgmasserrorPHREEQC]
Brmasserrorcompare=[Brmasserror BrmasserrorPHREEQC]
Clmasserrorcompare=[Clmasserror ClmasserrorPHREEQC]
timecompare=[tableautime PHREEQCtime]

Agcompare,1,2
1,5.53269e-08,5.61572e-08


Brcompare,1,2
1,9.05866e-06,9.05949e-06


Clcompare,1,2
1,3.16192e-05,3.16195e-05


AgBrcompare,1,2
1,9.21099e-07,9.20269e-07


AgClcompare,1,2
1,0,0


Agmasserrorcompare,1,2
1,0,3.47806e-09


Brmasserrorcompare,1,2
1,0,4.28484e-19


Clmasserrorcompare,1,2
1,1.07143e-13,-3.33399e-10


timecompare,1,2
1,8.71134,0.030581
